# North American Einstein Toolkit School 2021: NRPy+ Tutorial
## Authors: Leo Werneck, Zach Etienne, Thiago Assumpção, Terrence Pierre-Jacques, Patrick Nelson

<table><tr>
    <td style="background-color: white;">
        <a href=http://nrpyplus.net>
            <img src=https://github.com/leowerneck/NRPy_Tutorial_ETK_Workshop_2021/blob/main/logos/Nerpy.png?raw=true style="width: 150px;" />
        </a>
     </td>
    <td style="background-color: white;"> 
        <a href=https://www.wvu.edu>
            <img src=https://github.com/leowerneck/NRPy_Tutorial_ETK_Workshop_2021/blob/main/logos/WVU.png?raw=true style="width: 250px;"/>
        </a>
    </td>
    <td style="background-color: white;">
        <a href=https://www.uidaho.edu>
            <img src=https://github.com/leowerneck/NRPy_Tutorial_ETK_Workshop_2021/blob/main/logos/Idaho.png?raw=true style="width: 250px;"/>
        </a>
    </td>
    <td style="background-color: white;">
        <a href=https://einsteintoolkit.org>
            <img src=https://github.com/leowerneck/NRPy_Tutorial_ETK_Workshop_2021/blob/main/logos/ETK.png?raw=true style="width: 125px;"/>
        </a>
    </td>
    <td style="background-color: white;">
        <a href=https://www.nsf.gov>
            <img src=https://github.com/leowerneck/NRPy_Tutorial_ETK_Workshop_2021/blob/main/logos/NSF.jpg?raw=true style="width: 125px;"/>
        </a>
    </td>
</tr></table>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This tutorial notebook is organized as follows:

0. [Step 0](#introduction): Introduction
    1. [Step 0.a](#maxwell_equations): Maxwell's equations in vacuum, flat space, and Cartesian coordinates
    1. [Step 0.b](#potential_formulation_maxwell_equations): Potential formulation of Maxwell's equations
    1. [Step 0.c](#hyperbolicity_maxwell_equations): Improving the hyperbolicity of Maxwell's equations
    1. [Step 0.d](#systems_I_and_II): Systems I & II
1. [Step 1](#implementation): Part I - Implementation of the core equations in Curvilinear coordinates
    1. [Step 1.a](#initializenrpy): Initialize core Python/NRPy+ modules
    1. [Step 1.b](#equations): The equations
    1. [Step 1.c](#c_code_generation): Generating C code kernels from the symbolic expressions
1. [Step 2](#thorn_writing): Part II - Einstein Toolkit thorn writing
    1. [Step 2.a](#main_function): The thorn's driver/main function
    1. [Step 2.b](#ccl_files): Thorn configuration files
        1. [Step 2.b.i](#param_ccl): Generating the `param.ccl` file
        1. [Step 2.b.ii](#interface_ccl): Generating the `interface.ccl` file
        1. [Step 2.b.iii](#schedule_ccl): Generating the `schedule.ccl` file
        1. [Step 2.b.iv](#configuration_ccl): Generating the `configuration.ccl` file
        1. [Step 2.b.v](#make_code_defn): Generating the `make.code.defn` file
1. [Step 3](#results): Part III - Results
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='introduction'></a>

# Step 0: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

In this tutorial we will provide a hands-on introduction on how to use the [NRPy+ infrastructure](http://nrpyplus.net) to write thorns for the [Einstein Toolkit (ETK)](https://einsteintoolkit.org). Although we will give an overview of NRPy+ and provide a high level description of some of its modules, we do not aim to thoroughly describe every aspect of the infrastructure. For more details on NRPy+'s inner workings, we refer the reader to:

* [NRPy+'s extensive & pedagogical documentation](https://mybinder.org/v2/gh/zachetienne/nrpytutorial/master?filepath=NRPyPlus_Tutorial.ipynb)
* [NRPy+ tutorial from the 2020 ETK workshop by Zach Etienne](https://www.youtube.com/watch?v=TIPiW5-mPOM)

<hr style="width:100%;height:3px;color:black"/>

<a id='maxwell_equations'></a>

## Step 0.a: Maxwell's equations in vacuum, flat space, and Cartesian coordinates \[Back to [top](#toc)\]
$$\label{maxwell_equations}$$

In this tutorial we will generate an ETK thorn to solve [Maxwell's equations](https://en.wikipedia.org/wiki/Maxwell%27s_equations) in flat space using Cartesian coordinates. In [Gaussian](https://en.wikipedia.org/wiki/Gaussian_units) and $c = 1$ units, the system of equations we are interested in is

$$
\begin{align}
\vec{\nabla} \cdot \vec{E} &= 0, \\
\vec{\nabla} \cdot \vec{B} &= 0, \\
\frac{\partial \vec{E}}{\partial t} &= \vec{\nabla} \times \vec{B}, \\
\frac{\partial \vec{B}}{\partial t} &= -\vec{\nabla} \times \vec{E},
\end{align}
$$

where $\vec{E}$ is the electric field, $\vec{B}$ is the magnetic field, and $\vec{\nabla}\cdot\vec{V}$ and $\vec{\nabla}\times\vec{V}$ are the divergence and the curl of the vector $\vec{V}$, respectively.

The last two equations involve time derivatives of $\vec{E}$ and $\vec{B}$ and therefore are referred to as *evolution* equations. The first two equations must be satisfied for all times, and are referred to as *constraint* equations.

<hr style="width:100%;height:3px;color:black"/>

<a id='potential_formulation_maxwell_equations'></a>

## Step 0.b: Potential formulation of Maxwell's equations \[Back to [top](#toc)\]
$$\label{potential_formulation_maxwell_equations}$$

A formulation of Maxwell's equations that is particularly useful for numerical integration can be found by introducing auxiliary variables. We start by introducing a new vector quantity, $\vec{A}$, known as the *magnetic* or *vector* potential, such that

$$
\vec{B} = \vec{\nabla}\times\vec{A}.
$$

Note that the "no-magnetic monopole" constraint is automatically satisfied:

$$
\vec{\nabla} \cdot \vec{B} = \vec{\nabla} \cdot \bigl(\vec{\nabla}\times\vec{A}\bigr) = 0.
$$

[Ampère's law](https://en.wikipedia.org/wiki/Ampère%27s_circuital_law), which is the evolution equation for $\vec{B}$, can then be written as

$$
\frac{\partial}{\partial t}\bigl(\vec{\nabla}\times\vec{A}\bigr) = -\vec{\nabla} \times \vec{E} \implies \vec{\nabla}\times\left(\frac{\partial\vec{A}}{\partial t} + \vec{E}\right) = 0 \implies \frac{\partial\vec{A}}{\partial t} + \vec{E} = -\vec{\nabla}\phi,
$$

where $\phi$ is an arbitrary scalar function known as the *electric* (or *scalar*) potential. Finally, [Faraday's law](https://en.wikipedia.org/wiki/Faraday%27s_law_of_induction), the evolution equation for $\vec{E}$, can be written as

$$
\frac{\partial\vec{E}}{\partial t} = \vec{\nabla}\times\bigl(\vec{\nabla}\times\vec{A}\bigr) = -\nabla^{2}\vec{A} + \vec{\nabla}\bigl(\vec{\nabla}\cdot\vec{A}\bigr).
$$

Thus Maxwell's equations now read

$$
\begin{align}
\vec{\nabla} \cdot \vec{E} &= 0, \\
\frac{\partial\vec{E}}{\partial t} &= -\nabla^{2}\vec{A} + \vec{\nabla}\bigl(\vec{\nabla}\cdot\vec{A}\bigr),\\
\frac{\partial\vec{A}}{\partial t} &= -\vec{E} - \vec{\nabla}\phi.
\end{align}
$$

Note that we now end up with 7 dynamical fields, namely $\left(\phi,\vec{A},\vec{E}\right)$, but only 6 evolution equations. We can remedy this by adopting a particular gauge. We will choose here the [Lorenz gauge](https://en.wikipedia.org/wiki/Lorenz_gauge_condition), which reads

$$
\frac{\partial\phi}{\partial t} = -\vec{\nabla}\cdot\vec{A}.
$$

Thus we arrive at the evolution equations

$$
\boxed{
\begin{align}
\frac{\partial\vec{E}}{\partial t} &= -\nabla^{2}\vec{A} + \vec{\nabla}\bigl(\vec{\nabla}\cdot\vec{A}\bigr)\\
\frac{\partial\vec{A}}{\partial t} &= -\vec{E} - \vec{\nabla}\phi\\
\frac{\partial\phi}{\partial t} &= -\vec{\nabla}\cdot\vec{A}
\end{align}
}\ ,
$$

plus the constraints

$$
\boxed{ \vec{\mathcal{C}} \equiv \vec{\nabla} \cdot \vec{E} = 0}\ ,
$$

which must be satisfied at all times.

<hr style="width:100%;height:3px;color:black"/>

<a id='hyperbolicity_maxwell_equations'></a>

## Step 0.c: Improving the hyperbolicity of Maxwell's equations \[Back to [top](#toc)\]
$$\label{hyperbolicity_maxwell_equations}$$

If we take a time derivative of the evolution equation for the vector potential and plug in the evolution equation for the electric field and the scalar potential we find

$$
\frac{\partial^{2}\vec{A}}{\partial t^{2}} = -\frac{\partial\vec{E}}{\partial t} - \vec{\nabla}\left(\frac{\partial\phi}{\partial t}\right) = \nabla^{2}\vec{A} - \vec{\nabla}\bigl(\vec{\nabla}\cdot\vec{A}\bigr) - \vec{\nabla}\left(\frac{\partial\phi}{\partial t}\right).
$$

Notice that this is almost a wave equation for the vector potential $\vec{A}$, but we have a mixed derivative term on the right-hand side that spoils this behaviour. We can thus introduce a new auxiliary variable defined as

$$
\Gamma \equiv \vec{\nabla}\cdot\vec{A},
$$

so that we have

$$
\frac{\partial^{2}\vec{A}}{\partial t^{2}} = \nabla^{2}\vec{A} - \vec{\nabla}\Gamma - \vec{\nabla}\left(\frac{\partial\phi}{\partial t}\right).
$$

The resulting system of equations now becomes

$$
\boxed{
\begin{align}
\frac{\partial\vec{E}}{\partial t} &= -\nabla^{2}\vec{A} + \vec{\nabla}\Gamma\\
\frac{\partial\vec{A}}{\partial t} &= -\vec{E} - \vec{\nabla}\phi\\
\frac{\partial\phi}{\partial t} &= -\Gamma\\
\frac{\partial\Gamma}{\partial t} &= -\nabla^{2}\phi
\end{align}
}\ ,
$$

while the constraints that must satisfied are

$$
\boxed{
\begin{align}
\vec{\mathcal{C}} &\equiv \vec{\nabla} \cdot \vec{E} = 0\\
\mathcal{K} &\equiv \Gamma - \vec{\nabla}\cdot\vec{A} = 0
\end{align}
}\ .
$$

<hr style="width:100%;height:3px;color:black"/>

<a id='systems_I_and_II'></a>

## Step 0.d: Systems I & II \[Back to [top](#toc)\]
$$\label{systems_I_and_II}$$

Switching to index notation, we write

$$
\frac{\partial}{\partial t} \equiv \partial_{t}\ ;\ \frac{\partial}{\partial x^{i}} \equiv \partial_{i},
$$

where $\vec{x} \equiv x^{i} = (x,y,z)$. We thus have

$$
\text{System I:}\ 
\boxed{
\begin{align}
\partial_{t}E^{i} &= -\partial^{j}\partial_{j}A^{i} + \partial^{i}\partial_{j}A^{j}\\
\partial_{t}A^{i} &= -E^{i} - \partial^{i}\phi\\
\partial_{t}\phi  &= -\partial_{i}A^{i}\\
\partial_{i}E^{i} &= 0
\end{align}
}
\ ,
$$

and

$$
\text{System II:}\ 
\boxed{
\begin{align}
\partial_{t}E^{i} &= -\partial^{j}\partial_{j}A^{i} + \partial^{i}\Gamma\\
\partial_{t}A^{i} &= -E^{i} - \partial^{i}\phi\\
\partial_{t}\phi &= -\Gamma\\
\partial_{t}\Gamma &= -\partial^{i}\partial_{i}\phi\\
\partial_{i}E^{i} &= 0\\
\Gamma &= \partial_{i}A^{i}
\end{align}
}\ .
$$

<hr style="width:100%;height:3px;color:black"/>

<a id='implementation'></a>

# Step 1: Part I - Implementation of the core equations in Curvilinear coordinates \[Back to [top](#toc)\]
$$\label{implementation}$$

<a id='initializenrpy'></a>

## Step 1.a: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from NRPy+:

In [1]:
# Step 1.a: Import core Python/NRPy+ modules
# Step 1.a.i: Import needed Python modules
import shutil, os, sys # Standard Python modules for multiplatform OS-level functions

# Step 1.a.ii: Add the "nrpy_core" directory to the path
base_dir      = os.getcwd()
nrpy_core_dir = os.path.join(base_dir,"nrpy_core")
sys.path.append(nrpy_core_dir)

# Step 1.b: Load needed NRPy+ modules
import cmdline_helper as cmd # NRPy+: Multi-platform Python command-line interface

<a id='equations'></a>

## Step 1.b: The equations \[Back to [top](#toc)\]
$$\label{equations}$$

<a id='c_code_generation'></a>

## Step 1.c: Generating C code kernels from the symbolic expressions \[Back to [top](#toc)\]
$$\label{c_code_generation}$$

<hr style="width:100%;height:3px;color:black"/>

<a id='thorn_writing'></a>

# Step 2: Part II - Einstein Toolkit thorn writing \[Back to [top](#toc)\]
$$\label{thorn_writing}$$

<a id='main_function'></a>

## Step 2.a: The thorn's driver/main function \[Back to [top](#toc)\]
$$\label{main_function}$$

<a id='ccl_files'></a>

## Step 2.b: Thorn configuration files \[Back to [top](#toc)\]
$$\label{ccl_files}$$

<a id='param_ccl'></a>

### Step 2.b.i: Generating the `param.ccl` file \[Back to [top](#toc)\]
$$\label{param_ccl}$$

<a id='interface_ccl'></a>

### Step 2.b.ii: Generating the `interface.ccl` file \[Back to [top](#toc)\]
$$\label{interface_ccl}$$

<a id='schedule_ccl'></a>

### Step 2.b.iii: Generating the `schedule.ccl` file \[Back to [top](#toc)\]
$$\label{schedule_ccl}$$

<a id='configuration_ccl'></a>

### Step 2.b.iv: Generating the `configuration.ccl` file \[Back to [top](#toc)\]
$$\label{configuration_ccl}$$

<a id='make_code_defn'></a>

### Step 2.b.v: Generating the `make.code.defn` file \[Back to [top](#toc)\]
$$\label{make_code_defn}$$

<hr style="width:100%;height:3px;color:black"/>

<a id='results'></a>

# Step 3: Part III - Results \[Back to [top](#toc)\]
$$\label{results}$$

<hr style="width:100%;height:3px;color:black"/>

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[ETK_Workshop_2021-NRPy_tutorial.pdf](ETK_Workshop_2021-NRPy_tutorial.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [2]:
# Step 4: Generate a PDF version of this tutorial notebook
# Step 4.a: First copy the latex_nrpy_style.tplx from the
#           nrpy_core directory to the base directory
src_file = os.path.join(nrpy_core_dir,"latex_nrpy_style.tplx")
dst_file = os.path.join(base_dir     ,"latex_nrpy_style.tplx")
shutil.copyfile(src_file,dst_file)

# Step 4.b: Now generate the PDF file
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("ETK_Workshop_2021-NRPy_tutorial")

# Step 4.c: Clean up by removing the latex_nrpy_style.tplx from
#           the base directory
cmd.delete_existing_files(dst_file)

Created ETK_Workshop_2021-NRPy_tutorial.tex, and compiled LaTeX file to PDF
    file ETK_Workshop_2021-NRPy_tutorial.pdf
